# EcoSentinel: 基于ERNIE的森林保护巡视决策系统

## 一、项目背景

根据国家林业和草原局的公开信息和统计数据，我国大多省市的森林保护工作主要依赖人工筛查、响应，存在成本高、效率低和防护效果差的问题。
本项目基于飞桨文心大模型(ERNIE BOT)，依靠林业知识库进行“微调”训练，辅助使用 PaddleX套件的目标检测YOLO模型，构建 EcoSentinel —— 一个“**多模态信息-图语义分析-LLM-行动决策建议**”的端到端智慧森林巡视系统。

## 二、技术原理

这个项目分为两个主要任务。

第一个任务，使用 Salesforce 开源的 **BLIP** 模型，将上传的图像解析为描述性文本；利用百度飞桨的 ERNIE 模型配合定制知识库，分析图像解析得到的文本，生成专业林业管理员视角的专业行动建议。

该项目还结合了 飞桨PaddleX 进行目标检测，以便更精确地定位烟火源。这是我们的第二个任务。

整个工作流程通过 Gradio 提供了一个交互式的 GUI 应用。

![avatar](https://img-blog.csdnimg.cn/7ec55908bced420fbcfe55516948a975.png)

> 百度飞桨开源的ERNIE Bot(文心大模型)，是基于知识增强的持续学习语义理解的大语言模型。根据最新评估数据，文心大模型已经中文任务以及多项指标上超越了OpenAI GPT4.

> PaddleX是飞桨开源的全流程开发工具，集飞桨核心框架、模型库、工具及组件等深度学习开发所需全部能力于一身，并提供简明易懂的Python API，方便用户根据实际生产需求进行直接调用或二次开发，大幅降低了深度学习任务上手的门槛。

本项目代码已经开源，发布在Github上，点 👉[这里](https://github.com/chkhu/EcoSentinel)👈 访问

## 三、效果展示

本项目的介绍视频：点 👉[这里]()👈 跳转

<img src="https://img-blog.csdnimg.cn/13bb06c8bc95421bb8345f9a8642c8d0.jpeg" alt="ScreenShot.jpg" style="zoom:40%;" />

（计划后续将会部署到Hugging Face上。）


## 四、实现过程

## （二）任务一：大模型任务


对于大模型任务，我们首先使用salesforce开源的image2text-captioning BLIP模型，对我们上传的图像进行解析，返回一段描述文本。然后，拼接request，通过api向文心发出request，得到专业的林业管理员视角下的行动建议。

如果你希望直接运行python源代码，可以`python app/app_server.py` 直接运行我们提供的源代码（未嵌入视觉模块的版本）。




### 2.1 导入需要的库


以下是该项目需要使用的模块。若为安装，请根据log提示进行安装。


In [1]:
! pip install ipywidgets
! jupyter nbextension enable --py widgetsnbextension

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/, https://pypi.tuna.tsinghua.edu.cn/simple/

[notice] A new release of pip available: 22.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [2]:
import requests  # 用于发送 HTTP 请求
import json  # 用于处理 JSON 数据
import gradio as gr  # 用gradio创建图形用户界面
import io  # 用于处理字节流
from PIL import Image # 用于处理图像
import numpy as np 
import cv2

### 2.2 API和知识库的准备

#### 说明👇：

以text-concatenation的形式对ERNIE模型(文心大模型)进行“角色微调”。使用的“微调”材料采自我们搜集筛选的林业安全管理知识库。我们将这些材料和BLIP模型得到的描述文本进行拼接后作为我们的request发送给文心。

In [3]:
# 百度AI的API Key和Secret Key，用于链接文心大模型
SC_API_KEY = "*************************"
SC_SECRET_KEY = "*************************"

# 连接 salesforce BLIP 的inference API 准备
API_URL = "https://api-inference.huggingface.co/models/Salesforce/blip-image-captioning-large"
blip_headers = {"Authorization": "Bearer api_org_*************************"}

# 自定义的知识库字符串，用于个性化定制文心大模型
knowledge_base = ""  

# 为应对多次分析请求，初始化一个列表来存储所有生成的文本。
generated_texts = []

### 2.3 功能函数的定义

In [4]:
# 文心API的调用函数
# INPUT：用户的输入文本
# OUTPUT：文心的回答文本
def main_app(input_text):
    # 拼接 URL，文心的base_url + access token, 只有拥有有效的access_token，才能成功调用文心API。
    # get_access_token()函数会发送一个 HTTP POST 请求到百度 获取access_token。 具体的实现在下面。
    url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/completions?access_token=" + get_access_token()
    payload = json.dumps({  # 将 Python 对象转换编码成 JSON 字符串
        "messages": [   # 拼接出完整query，包含三部分：引导词、知识库、场景描述
            {
                "role": "user",
                "content": "假如你是一个专业林业安全管理员，熟知：" + knowledge_base + 
                    "要根据下面的描述给出一个解决方案，并且回答中只需含有解决方案：" + 
                    input_text,
            }
        ]
    })
    headers = { # 设置 HTTP 请求的头部信息
        'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload) # 发送 HTTP POST 请求，返回一个response对象
    result = response.json().get("result")  # 从response对象中提取出 JSON 数据（回答文本）
    print("WENXIN RESPONSE:\n" + result + "\n")
    return result

# 获取百度AI的access_token
# INPUT：无，但需要使用到前面定义的 SC_API_KEY 和 SC_SECRET_KEY
# OUTPUT：access_token
def get_access_token(): 
    url = f"https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={SC_API_KEY}&client_secret={SC_SECRET_KEY}" # 拼接 request URL
    payload = ""
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload) # 发送 HTTP POST 请求，返回一个response对象
    print("ACCESS_TOKEN OBTAINED: \n" + response.text + "\n")
    return str(response.json().get("access_token"))


#### 说明👇：

我们将会使用BLIP模型实现图生文任务。

> BLIP 是由Salesforce开发的一个非常强大的视觉语义理解模型,官方提供了hugging face workspace和了非常便捷的off-the-shelf api服务,帮助开发者快速应用这一模型能力。
> 本项目中通过inference API的方式调用BLIP模型。

In [5]:
# BLIP API的调用函数
def query(image_data):
    response = requests.post(API_URL, headers=blip_headers, data=image_data)
    return response.json()

def judge_query(image_data):
    response = requests.post(API_URL_VIT, headers=headers_VIT, data=image_data)
    return response.json()

def judge(input_image):
    # 将输入的input_image(numpy数组类型)转成query函数可以接受的Bytes类型
    img = Image.fromarray(input_image.astype(np.uint8))  # 需要转换为 uint8，范围 [0, 255]    
    img_byte_arr = io.BytesIO()  # 创建一个字节流对象
    img.save(img_byte_arr, format='PNG')  # 将图像保存到字节流中，可以选择不同的格式，例如 'JPEG'，这里用 'PNG'
    img_byte_arr = img_byte_arr.getvalue()  # 获取字节流的值

    output = judge_query(img_byte_arr)

    # # 检查输出是否包含错误信息
    # if 'error' in output:
    #     # 处理错误情况，例如返回错误信息或重试请求
    #     return f"Error: {output['error']}"

    # # DEBUG
    print("✨✨✨✨✨✨✨✨✨VIT output:", output)

    # 如果没有错误，则格式化输出
    formatted_output = "烟火元素置信度分析结果：\n"

    # 提取出output中label值为Normal、Fire、Smoke的置信度
    for item in output:
        if item['label'] == 'Normal':
            normal_score = item['score']
            formatted_output += f"Normal置信度: {item['score']}\n"
        elif item['label'] == 'Fire':
            fire_score = item['score']
            formatted_output += f"Fire置信度: {item['score']}\n"
        elif item['label'] == 'Smoke':
            smoke_score = item['score']
            formatted_output += f"Smoke置信度: {item['score']}\n"

    formatted_output += "\n烟火目标判断结果：\n"
    if fire_score > normal_score or smoke_score > normal_score:
        formatted_output += "⚠️存在火情"
    else:
        formatted_output += "🍃不存在火情"

    judgement_output = formatted_output

    # 最后，返回生成的文本
    return judgement_output



# 图片分析函数
# INPUT：用户上传的图片
# OUTPUT：图片分析文本
def image_analysis(input_image):
    # # DEBUG: 保存input_image到本地
    # cv2.imwrite('temp.jpg', input_image)
    # print("IMAGE ARRAY MAX VALUE" + str(input_image.max())) # 255

    # 将输入的input_image(numpy数组类型)转成query函数可以接受的Bytes类型
    img = Image.fromarray(input_image.astype(np.uint8))  # 需要转换为 uint8，范围 [0, 255]    
    img_byte_arr = io.BytesIO()  # 创建一个字节流对象
    img.save(img_byte_arr, format='PNG')  # 将图像保存到字节流中，可以选择不同的格式，例如 'JPEG'，这里用 'PNG'
    img_byte_arr = img_byte_arr.getvalue()  # 获取字节流的值

    # 调用 query 函数
    output = query(img_byte_arr)

    # # DEBUG
    # print("TYPE OF OUTPUT OF QUERY\n")
    # print(type(output)) # list
    
    # 取出列表的0号元素，然后用 ['generated_text'] 取出字典中的值
    newly_generated_text = output[0]['generated_text'] 
    generated_texts.append(newly_generated_text)
    generated_text = generated_texts[-1] #  取列表的最后一个元素，确保每次都是最新的文本

    # 最后，返回生成的文本
    return generated_text

我们对视频的处理：将其分片并抽取几帧，对这些帧逐一分析：

In [ ]:
def video_analysis(input_video):
    # 思路是抽取视频关键帧，然后对关键帧进行分析，最后将所有的分析结果拼接起来
    # 1. 读取视频
    vidcap = cv2.VideoCapture(input_video)
    # 获取视频总帧数
    vidFrame = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))  
    # 视频时间
    if(vidFrame==0):
        return "视频无效，请重新上传！"
    vidTime = vidFrame / vidcap.get(cv2.CAP_PROP_FPS)
    maxFrame = 10
    minInterval = 0.5 
    # 实际每几秒处理一次
    intervalTime = max(minInterval, vidTime / maxFrame)
    # 实际每几帧处理一次
    intervalFrame = int(intervalTime * vidcap.get(cv2.CAP_PROP_FPS))

    # 2. 抽取帧处理
    success, image = vidcap.read()
    count = 0

    video_analysis_result = []
    while success:
        # 2.1 处理帧
        result_of_frame = image_analysis(image)
        video_analysis_result.append(result_of_frame)
        # 2.2 读取下一帧
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, count * intervalFrame)
        success, image = vidcap.read()
        count += 1
        print("FRAME " + str(count) + " ANALYZED: "+ result_of_frame)
    # 3. 拼接结果
    return "\n".join(video_analysis_result)

### Gradio搭建用户界面

最后我们基于graddio封装了我们一系列模型的工作管线，提供了简单易用可交互的GUI应用。

In [6]:
### Gradio UI ###
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Tab("上传图片"):
            image_input = gr.Image()
            img_analysis_button = gr.Button("分析图片")
        with gr.Tab("上传视频"):
            video_input = gr.Video()
            video_analysis_button = gr.Button("分析视频")
        analysis_result = gr.Textbox(label="分析结果")
        
        
    with gr.Tab("建议"):
        text_output = gr.Textbox(label="输出")
    text_button = gr.Button("获取建议")

    with gr.Tab("烟火元素置信度分析(适用林火)"):  # 新增Row
        judge_image_output = gr.Textbox(label="分析判断结果", image_mode='fixed', width=600)
    judge_button = gr.Button("分析判断图片")
        
    with gr.Tab("烟火目标检测可视化(适用林火)"):  # 新增Row
        detected_image_output = gr.Image(label="目标检测结果", image_mode='fixed', width=600)  # 新增图像输出组件
    detect_button = gr.Button("显示目标检测结果")  # 新增按钮
    
    img_analysis_button.click(image_analysis, inputs=image_input, outputs=analysis_result) # 将按钮与image_analysis函数关联
    video_analysis_button.click(video_analysis, inputs=video_input, outputs=analysis_result) # 将按钮与video_analysis函数关联

    text_button.click(main_app, inputs=analysis_result, outputs=text_output) # 将按钮与main_app函数关联
    
    detect_button.click(detect, inputs=image_input, outputs=detected_image_output)  # 将按钮与detect函数关联
    judge_button.click(judge, inputs=image_input, outputs=judge_image_output)  # 将按钮与judge_query函数关联

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


你应该可以预期看到类似下面的界面。

<img src="https://img-blog.csdnimg.cn/a0ba826f892d4da38c41c982e840ef0d.png" alt="base_gui.png" style="zoom:40%;" />

正常情况下，上面的log将会显示本地预览demo的接口，点击即可进行测试。

值得指出的是，我们的项目允许用户根据实际需要，**优化recompose编辑BLIP生成的输出captioning**，来应对描述质量不够稳定的问题。这一过程在演示视频的demo中有展示。

## （二）任务二：视觉任务


为了实现对火灾场景监测效果的进一步优化，我们为林火场景特别引入了百度飞桨开源套件PaddleX的目标检测模型YOLO，对输入航拍中的烟火进行检测定位，为林业消防部门提供精确的指引。部分操作参考基于AI studio开放项目“PaddleX的火焰检测与识别”。

### 2.1 安装PaddleX


本任务要求PaddleX的任务不超过2.0.0，因此在命令中强制版本。

In [7]:
! pip install scikit-learn

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/, https://pypi.tuna.tsinghua.edu.cn/simple/

[notice] A new release of pip available: 22.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [8]:
! pip install "paddlex<2.0.0" -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple, https://mirrors.aliyun.com/pypi/simple/, https://pypi.tuna.tsinghua.edu.cn/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.0/517.0 kB 5.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.3/145.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 10.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.3/403.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: paddlehub
    Found existing installation: paddlehub 2.0.4
    Uninstalling paddlehub-2.0.4:
      Successfully uninstalled paddlehub-2.0.4

[notice] A new release of pip available: 22.1.2 -> 23.2.1
[notice] To update, run: pip install --upgra

### 2.2 数据集

实验过程中，我们比对测试了开放数据集和自建数据集的效果，最终选择了下面这个数据集。

#### 2.2.1 解压并查看数据集。


In [19]:
# ! unzip -oq /home/aistudio/data/fire.zip -d /home/aistudio/data/
# ! tree /home/aistudio/data/fire -L 2

### 2.2.2 数据集划分
运行命令，在指定的目录（home/aistudio//fire）下寻找 VOC 格式的数据集，并将其划分为三个子集：训练集、验证集和测试集。

划分比例是：训练集占 70%（100% - 20% - 10%），验证集占 20%，测试集占 10%。

In [20]:
! pip install chardet

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/, https://pypi.tuna.tsinghua.edu.cn/simple/

[notice] A new release of pip available: 22.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [21]:
import paddle
print(paddle.__version__)

2.1.2


In [23]:
# paddlex 自带数据集划分工具
! paddlex --split_dataset --format VOC --dataset_dir /home/aistudio/data/fire --val_value 0.2 --test_value 0.1

Dataset Split Done.
Train samples: 345
Eval samples: 98
Test samples: 49
Split files saved in /home/aistudio/data/fire


### 2.3 模型训练

#### 2.3.1 预先配置

导入需要的工具包，包括PaddleX套件。

In [26]:
# 默认使用0号GPU卡（无GPU情况将会调用cpu训练）
import matplotlib
matplotlib.use('Agg') 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import paddlex as pdx
print(pdx.__version__)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://matplotlib_inline.backend_inline' by the following code:
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/traitlets/config/application.py", line 978, in launch_instance
    app.start()
  File "/opt/conda/env

1.3.11


定义数据处理流程，其中训练和测试需分别定义，训练过程包括了部分测试过程中不需要的数据增强操作，如在本示例中，训练过程使用了`MixupImage`、`RandomDistort`、`RandomExpand`、`RandomCrop`和`RandomHorizontalFlip`共5种数据增强方式。

In [27]:
import paddlex as pdx
from paddlex.det import transforms
train_transforms = transforms.Compose([
    transforms.MixupImage(mixup_epoch=250),
    transforms.RandomDistort(),
    transforms.RandomExpand(),
    transforms.RandomCrop(),
    transforms.Resize(target_size=608, interp='RANDOM'),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(),
])

# 验证集部分只需要resize和normalize即可
eval_transforms = transforms.Compose([
    transforms.Resize(target_size=608, interp='CUBIC'),
    transforms.Normalize(),
])

#### 2.3.2 指定训练数据路径

首先查看目前的数据集目录的清单。

In [28]:
! ls /home/aistudio/data/fire

Annotations  labels.txt     train_list.txt
JPEGImages   test_list.txt  val_list.txt


由于数据集为VOC格式，因此采用`pdx.datasets.VOCDetection`来加载数据集

In [30]:
# 此处的数据集路径全部采用绝对路径

train_dataset = pdx.datasets.VOCDetection(
    data_dir='/home/aistudio/data/fire',
    file_list='/home/aistudio/data/fire/train_list.txt',
    label_list='/home/aistudio/data/fire/labels.txt',
    transforms=train_transforms,
    shuffle=True)
eval_dataset = pdx.datasets.VOCDetection(
    data_dir='/home/aistudio/data/fire',
    file_list='/home/aistudio/data/fire/val_list.txt',
    label_list='/home/aistudio/data/fire/labels.txt',
    transforms=eval_transforms)

2023-09-29 20:09:21 [INFO]	Starting to read file list from dataset...
2023-09-29 20:09:21 [INFO]	345 samples in file /home/aistudio/data/fire/train_list.txt
creating index...
index created!
2023-09-29 20:09:21 [INFO]	Starting to read file list from dataset...
2023-09-29 20:09:21 [INFO]	98 samples in file /home/aistudio/data/fire/val_list.txt
creating index...
index created!


#### 2.3.3 开始训练

使用本数据集V100训练，需要约15分钟。模型训练过程每间隔save_interval_epochs轮次会保存一次模型在save_dir目录下，同时在保存的过程中也会在验证数据集上计算相关指标。

你可以根据自己的需要调整 epoch, interval, lr 等参数。最后的训练耗时与模型效果将会有所差异。


In [31]:
num_classes = len(train_dataset.labels)
model = pdx.det.YOLOv3(num_classes=num_classes, backbone='DarkNet53')
model.train(
    num_epochs=40,
    train_dataset=train_dataset,
    train_batch_size=12,
    log_interval_steps=28,
    eval_dataset=eval_dataset,
    learning_rate=0.000125,
    lr_decay_epochs=[210, 240],
    save_interval_epochs=10,
    early_stop=True,
    save_dir='output/yolov3_darknet53',
    use_vdl=True)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/framework.py:706: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:


2023-09-29 20:09:28 [INFO]	Downloading DarkNet53_ImageNet1k_pretrained.tar from https://paddle-imagenet-models-name.bj.bcebos.com/DarkNet53_ImageNet1k_pretrained.tar


100%|██████████| 162940/162940 [00:06<00:00, 23293.95KB/s]


2023-09-29 20:09:35 [INFO]	Decompressing output/yolov3_darknet53/pretrain/DarkNet53_ImageNet1k_pretrained.tar...


W0929 20:09:35.996111   185 device_context.cc:404] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 10.1
W0929 20:09:36.001534   185 device_context.cc:422] device: 0, cuDNN Version: 7.6.


...

2023-09-29 20:16:47 [INFO]	[EVAL] Finished, Epoch=20, bbox_map=2.721541 .
2023-09-29 20:16:48 [INFO]	Model saved in output/yolov3_darknet53/epoch_20.
2023-09-29 20:16:48 [INFO]	Current evaluated best model in eval_dataset is epoch_10, bbox_map=3.03030303030303
2023-09-29 20:17:08 [INFO]	[TRAIN] Epoch=21/40, Step=28/28, loss=14.668419, lr=7.3e-05, time_each_step=0.48s, eta=0:6:49
2023-09-29 20:17:08 [INFO]	[TRAIN] Epoch 21 finished, loss=15.361101, lr=7.2e-05 .
2023-09-29 20:17:27 [INFO]	[TRAIN] Epoch=22/40, Step=28/28, loss=16.022993, lr=7.7e-05, time_each_step=0.46s, eta=0:6:18
2023-09-29 20:17:27 [INFO]	[TRAIN] Epoch 22 finished, loss=15.28491, lr=7.5e-05 .
2023-09-29 20:17:48 [INFO]	[TRAIN] Epoch=23/40, Step=28/28, loss=14.622307, lr=8e-05, time_each_step=0.46s, eta=0:5:47
2023-09-29 20:17:48 [INFO]	[TRAIN] Epoch 23 finished, loss=14.947395, lr=7.9e-05 .
2023-09-29 20:18:08 [INFO]	[TRAIN] Epoch=24/40, Step=28/28, loss=16.607906, lr=8.4e-05, time_each_step=0.47s, eta=0:5:49
2023-09-2

100%|██████████| 9/9 [00:05<00:00,  1.51it/s]


2023-09-29 20:20:19 [INFO]	[EVAL] Finished, Epoch=30, bbox_map=8.899207 .
2023-09-29 20:20:23 [INFO]	Model saved in output/yolov3_darknet53/best_model.
2023-09-29 20:20:24 [INFO]	Model saved in output/yolov3_darknet53/epoch_30.
2023-09-29 20:20:24 [INFO]	Current evaluated best model in eval_dataset is epoch_30, bbox_map=8.89920740617489
2023-09-29 20:20:44 [INFO]	[TRAIN] Epoch=31/40, Step=28/28, loss=13.893236, lr=0.000108, time_each_step=0.44s, eta=0:3:20
2023-09-29 20:20:44 [INFO]	[TRAIN] Epoch 31 finished, loss=12.884614, lr=0.000107 .
2023-09-29 20:21:04 [INFO]	[TRAIN] Epoch=32/40, Step=28/28, loss=13.20516, lr=0.000112, time_each_step=0.51s, eta=0:2:52
2023-09-29 20:21:04 [INFO]	[TRAIN] Epoch 32 finished, loss=12.71328, lr=0.00011 .
2023-09-29 20:21:25 [INFO]	[TRAIN] Epoch=33/40, Step=28/28, loss=10.538862, lr=0.000115, time_each_step=0.51s, eta=0:2:30
2023-09-29 20:21:25 [INFO]	[TRAIN] Epoch 33 finished, loss=12.870336, lr=0.000114 .
2023-09-29 20:21:47 [INFO]	[TRAIN] Epoch=34/40

100%|██████████| 9/9 [00:08<00:00,  1.09it/s]


2023-09-29 20:24:03 [INFO]	[EVAL] Finished, Epoch=40, bbox_map=33.137789 .
2023-09-29 20:24:07 [INFO]	Model saved in output/yolov3_darknet53/best_model.
2023-09-29 20:24:09 [INFO]	Model saved in output/yolov3_darknet53/epoch_40.
2023-09-29 20:24:09 [INFO]	Current evaluated best model in eval_dataset is epoch_40, bbox_map=33.13778888751509


### 2.4 评估与模型预测

基于已经训练好的模型，我们对样本图像进行预测

In [34]:
import paddlex as pdx
model = pdx.load_model('output/yolov3_darknet53/best_model')
image_name = '/home/aistudio/test_image.jpg'
result = model.predict(image_name)
# result
# 将result 进行可视化，并保存到指定路径
pdx.det.visualize(image_name, result, threshold=0.05, save_dir='/home/aistudio')

2023-09-29 20:29:38 [INFO]	Model[YOLOv3] loaded.
2023-09-29 20:29:38 [INFO]	The visualized result is saved as /home/aistudio/visualize_test_image.jpg


### 2.5 整合进入基础gradio界面

定义检测函数。这依赖于已经训练好的现成的best_model。

In [36]:
import paddlex as pdx
from paddlex.det import transforms

# 目标检测函数
# INPUT：用户上传的图片
# OUTPUT：图片经目标检测的返图
def detect(input_image):
    
    # FILL TEMPORAIRLY to test the front-end look
    output_image = Image.open('visualize_test_image.jpg')

    return output_image

修改Gradio的设计布局，加入面向林火监测的烟火目标检测模块。（为避免混淆，将之前 **无yolo版本**部署的网页的后台终端，按`^C`终止。）

In [37]:
# 在Gradio UI中增加新的组件
with gr.Row():
        with gr.Tab("上传图片"):
            image_input = gr.Image()
            img_analysis_button = gr.Button("分析图片")
        with gr.Tab("上传视频"):
            video_input = gr.Video()
            video_analysis_button = gr.Button("分析视频")
        analysis_result = gr.Textbox(label="分析结果")
        
        
    with gr.Tab("建议"):
        text_output = gr.Textbox(label="输出")
    text_button = gr.Button("获取建议")

    with gr.Tab("烟火元素置信度分析(适用林火)"):  # 新增Row
        judge_image_output = gr.Textbox(label="分析判断结果", image_mode='fixed', width=600)
    judge_button = gr.Button("分析判断图片")
        
    with gr.Tab("烟火目标检测可视化(适用林火)"):  # 新增Row
        detected_image_output = gr.Image(label="目标检测结果", image_mode='fixed', width=600)  # 新增图像输出组件
    detect_button = gr.Button("显示目标检测结果")  # 新增按钮
    
    img_analysis_button.click(image_analysis, inputs=image_input, outputs=analysis_result) # 将按钮与image_analysis函数关联
    video_analysis_button.click(video_analysis, inputs=video_input, outputs=analysis_result) # 将按钮与video_analysis函数关联

    text_button.click(main_app, inputs=analysis_result, outputs=text_output) # 将按钮与main_app函数关联
    
    detect_button.click(detect, inputs=image_input, outputs=detected_image_output)  # 将按钮与detect函数关联
    judge_button.click(judge, inputs=image_input, outputs=judge_image_output)  # 将按钮与judge_query函数关联

demo.launch()

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/gradio/deprecation.py:40: UserWarning: `width` is deprecated in `Interface()`, please use it within `launch()` instead.
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


正常情况，访问生成的预览url你将看到类似下面的界面。

<img src="https://img-blog.csdnimg.cn/dd881aec57db4387a70c43ea3d4eb2f4.png" alt="withYOLO_gui.png" style="zoom:50%;" />

你可以使用自己图片进行自由测试。

## 五、总结

我们使用 Salesforce 开源的 **BLIP** 模型，将上传的图像解析为描述性文本；利用百度飞桨的 ERNIE 模型配合定制知识库，分析图像解析得到的文本，生成专业林业管理员视角的专业行动建议。该项目还结合了 飞桨PaddleX 进行目标检测标记，以便更精确地定位烟火源。

该项目预期为林业安全管理起到高效的辅助作用。

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 